# Interactive Visualisation

### Objective:
We aim to visualize the distribution of grants issued to different Swiss Cantons by Swiss National Science Foundation. To achieve this, grant's data is collected from the P3 Grant Export, interpreted and presented into a topographic map.

In [1]:
# A number of libraries are need
import numpy as np
import pandas as pd
import folium
import json
from urllib.request import urlopen
from urllib import parse
import requests

#### Cleaning data
First we load and clean the file, keeping only the information about the monetary amount of funding as well as the university they belong to. The canton it is not directly provided with the dataset, so further work will be put later into mapping universities to cantons.

The database description mentions that the University Field is left **empty** if the project is not carried out in Swiss University. Hence we decide to drop these entries as the money is not used by the Swiss University. There were **12981** such enteries. Furthermore, 'Nicht zuteilbar - NA' is not really an university, so it is excluded too.

Funding is often missing too: In these cases our approach is to set the amount to 0 so it will be aggregated into cantons without an impact in our data. This behaviour is easily modifiable in the code below. There are **10990** such projects.

In [2]:
# Loading from csv file on root directory
df = pd.read_csv('P3_GrantExport.csv',sep=';')
data = df.copy()

# Dropping innecessary information
df = df[['University','Approved Amount']]

# Transformation from string to floats in 'Approved Amount' while also dealing with non-numeric values.
float_amounts = list()
for x in df['Approved Amount'].tolist():
    try:
        float_amounts.append(float(x))
    except:
        float_amounts.append(0) #When we are missing a real number we set the amount to 0 (basically to discard it later)
    
df = df[['University']] #we keep only university names associated at projects
df['Funding'] = float_amounts #and we attach float value of the Approved Amount, now renamed to 'Funding'

# Final manual drops
df = df.drop(df.index[(df['University'] ==  'Nicht zuteilbar - NA')])
df = df.drop(df.index[(df['University'] ==  'NPO (Biblioth., Museen, Verwalt.) - NPO')])

# Contains data for NPO
npo_data = data[data['University'] == 'NPO (Biblioth., Museen, Verwalt.) - NPO']

df.head()

,University,Funding
1,Université de Genève - GE,41022.0
3,Universität Basel - BS,52627.0
5,Université de Fribourg - FR,53009.0
6,Université de Fribourg - FR,25403.0
7,Universität Zürich - ZH,47100.0


Before continuing with the map to cantons, to drastically reduce the number of requests, we can group by university simply accumalating the funding of every project conceded to them.

In [3]:
funding_by_uni = df.groupby('University')[['Funding']].sum()
funding_by_uni.describe()

,Funding
count,7.500000e+01
mean,1.666842e+08
std,4.449057e+08
min,8.000000e+03
25%,1.325184e+06
50%,5.067172e+06
75%,4.010541e+07
max,1.838237e+09


75 different universities with a huge disparity in funding.
Maximum is 3 orders of magnitude bigger than the 1st quantile's top.

#### Expanding data
Now that we have reduced our data to the minimum useful amount, it's time to add some new fields that enhance our capabilities.

We expand our columns to include the Canton and, as an extra, also the longitude and latitude to visualize our universities geolocated in the map.
In an older version, we also notice that the university field can be splited into name of the university and code. This approach helped us to achieve a higher percentage of correct geo-searchs later on, but incurred in some unwanted matches like University of Laussane being associated to ZH! Our final approach simply removes "-" and "(" when forming the query.

In [4]:
# Extra fields
funding_by_uni['Longitude'] = ''; funding_by_uni['Latitude'] = ''; funding_by_uni['Canton'] = ''

#imporving for before search
fbu = funding_by_uni.reset_index()

# Old version expansion
'''
University_name = list()
University_code = list()
for x in fbu.University.tolist():
    try:
        name, code = x.split(' - ') #split by line
    except:
        name, code = x.split(' ') #or split by space (there are some cases in which the split is different)
        
    University_name.append(name)
    University_code.append(code)
fbu['University_name'] = University_name
fbu['University_code'] = University_code

'''

fbu.head()
    

,University,Funding,Longitude,Latitude,Canton
0,AO Research Institute - AORI,3435621.0,,,
1,Allergie- und Asthmaforschung - SIAF,19169965.0,,,
2,Berner Fachhochschule - BFH,31028695.0,,,
3,Biotechnologie Institut Thurgau - BITG,2492535.0,,,
4,Centre de rech. sur l'environnement alpin - CR...,1567678.0,,,


Now we use Geonames Full Text Search API to map the universities to their respective cantons. 'requests' library is used to get the HTTP response. But since 'University' names have spaces and other special characters, it needs to be UTF-8 encoded before using the requests.get. This is accomplished using 'parse.quote' method in 'urllib' library.

In [5]:
num_projects = len(fbu)
username = 'dunaiada'

for i in range(0,num_projects):
    
    # Cleaning unnecessary characters for query
    uni_tmp = fbu['University'][i]
    if uni_tmp.find('-') != -1:
        uni_tmp = uni_tmp.replace(uni_tmp[uni_tmp.find('-'):],'')
    if uni_tmp.find('(') != -1:
        uni_tmp = uni_tmp.replace(uni_tmp[uni_tmp.find('('):],'')
        
    # Restricting to country=CH is of critical importance, particularly when usid the code
    url = 'http://api.geonames.org/search?q=' + uni_tmp + '&country=CH' + '&maxRows=1&username=' + username + '&type=json'
    
    ## Encode special characters and spaces
    query = parse.quote(url,safe=':/&=?')
    
    ## Parse JSON data
    d = json.loads(requests.get(query).text)
    
    if bool(d['geonames']): #checks whether we get a match.
        #And because you don't always get all the info, a try-except is needed
        try:
            fbu['Canton'].iloc[i]     = d['geonames'][0]['adminCode1']
            fbu['Latitude'].iloc[i]   = d['geonames'][0]['lat']
            fbu['Longitude'].iloc[i]  = d['geonames'][0]['lng']
        except:
            pass

/Users/nesara/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
fbu.head(10)

,University,Funding,Longitude,Latitude,Canton
0,AO Research Institute - AORI,3.435621e+06,,,
1,Allergie- und Asthmaforschung - SIAF,1.916996e+07,,,
2,Berner Fachhochschule - BFH,3.102870e+07,,,
3,Biotechnologie Institut Thurgau - BITG,2.492535e+06,,,
4,Centre de rech. sur l'environnement alpin - CR...,1.567678e+06,,,
5,EPF Lausanne - EPFL,1.175316e+09,,,
6,ETH Zürich - ETHZ,1.635597e+09,8.54805,47.3763,ZH
7,Eidg. Anstalt für Wasserversorgung - EAWAG,7.461922e+07,,,
8,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",4.836039e+07,,,
9,Eidg. Hochschulinstitut für Berufsbildung - EHB,2.086572e+06,,,


We can see that despite our efforts, still a great number of universities hadn't been associated to their respective cantons. (Not even our lovely EPFL). To attempt to solve this we recurre to a different library, geocoder, that allow us to tap into the more powerful google maps API.

In [ ]:
import geocoder

num_projects = len(fbu)
for i in range(0,num_projects):
    
    # Cleaning unnecessary characters for query
    uni_tmp = fbu['University'][i]
    if uni_tmp.find('-') != -1:
        uni_tmp = uni_tmp.replace(uni_tmp[uni_tmp.find('-'):],'')
    if uni_tmp.find('(') != -1:
        uni_tmp = uni_tmp.replace(uni_tmp[uni_tmp.find('('):],'')
        
    
    # Here we use google maps' access
    response = geocoder.google(uni_tmp,components="country:CH")
    json_out = response.json
    
    # If the canton is found, it will appear as 'state' in the JSON
    if 'state' in json_out.keys():
        fbu['Canton'].loc[i]    = json_out['state']
        fbu['Latitude'].loc[i]  = json_out['lat']
        fbu['Longitude'].loc[i] = json_out['lng']

In [79]:
fbu.head(10)

,University,Funding,Longitude,Latitude,Canton
0,AO Research Institute - AORI,3.435621e+06,,,GR
1,Allergie- und Asthmaforschung - SIAF,1.916996e+07,,,GR
2,Berner Fachhochschule - BFH,3.102870e+07,,,BE
3,Biotechnologie Institut Thurgau - BITG,2.492535e+06,9.05574,47.6038,TG
4,Centre de rech. sur l'environnement alpin - CR...,1.567678e+06,,,ZH
5,EPF Lausanne - EPFL,1.175316e+09,6.5676,46.519,VD
6,ETH Zürich - ETHZ,1.635597e+09,8.54809,47.3765,ZH
7,Eidg. Anstalt für Wasserversorgung - EAWAG,7.461922e+07,,,SG
8,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",4.836039e+07,,,TI
9,Eidg. Hochschulinstitut für Berufsbildung - EHB,2.086572e+06,,,ZH


In [84]:
map_with_uni = folium.Map(location=[46.76, 8.26], tiles='Mapbox Bright',zoom_start=8)
for indx in range(fbu.shape[0]):
    if fbu.iloc[indx]['Latitude'] != ''  :
        folium.Marker(location=[fbu.iloc[indx]['Latitude'],fbu.iloc[indx]['Longitude']],
                      popup=fbu.iloc[indx]['University']).add_to(map_with_uni)
map_with_uni

There is some improvement. At least we (EPFL) are there now. And if we actually consider how much money had been correctly allocated, missing universities are just small amounts compared to EPFL and ETHZ.

Still, because we want to have an exhaustive represantition, we have taken the time to manually collect the missing cantons. For this, a CSV file had been created with the new inputs. We will work with such file from the last steps of the exercise.

In [75]:
manually_mapped_data = pd.read_csv('missing_mappings.csv',sep=',',encoding = 'cp1252')

In [12]:
val =  fbu.University.isin(manually_mapped_data.University)
fbu.loc[val,'Canton'] = manually_mapped_data['Canton']
fbu.head(10)

,University,Funding,Longitude,Latitude,Canton
0,AO Research Institute - AORI,3.435621e+06,,,GR
1,Allergie- und Asthmaforschung - SIAF,1.916996e+07,,,GR
2,Berner Fachhochschule - BFH,3.102870e+07,,,BE
3,Biotechnologie Institut Thurgau - BITG,2.492535e+06,9.05574,47.6038,TG
4,Centre de rech. sur l'environnement alpin - CR...,1.567678e+06,,,ZH
5,EPF Lausanne - EPFL,1.175316e+09,6.5676,46.519,VD
6,ETH Zürich - ETHZ,1.635597e+09,8.54809,47.3765,ZH
7,Eidg. Anstalt für Wasserversorgung - EAWAG,7.461922e+07,,,SG
8,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",4.836039e+07,,,TI
9,Eidg. Hochschulinstitut für Berufsbildung - EHB,2.086572e+06,,,ZH


Grouping by cantons and including non-represente cantons is the last step of processing in our data

In [101]:
fbc = fbu.groupby('Canton')[['Funding']].sum() #fbc == funding by canton
data = fbc.reset_index()

list_of_cantons = ["ZH","BE","LU","UR", "SZ", "OW","NW", "GL", "ZG", "FR", "SO", "BS", "BL", "SH", "AR", "AI",
                   "SG", "GR", "AG", "TG", "TI", "VD", "VS", "NE", "GE", "JU"]

frame_of_cantons = pd.DataFrame(list_of_cantons, columns=['Canton']) #Creates a dataframe from the list to allow merging
extended_data = pd.merge(frame_of_cantons, data, how='left', on="Canton")
extended_data.head()

,Canton,Funding
0,ZH,3.473426e+09
1,BE,1.552404e+09
2,LU,5.971181e+07
3,UR,NaN
4,SZ,9.365510e+05


In [21]:
npo_data_ = pd.DataFrame(npo_data.groupby('Institution').apply(lambda x: x['Approved Amount'].astype(float).sum()))
npo_data_.columns = ['Funding']
npo_data_['Canton'] = ''

#import geocoder
for indx in npo_data_.index :
    
    response = geocoder.google(indx,components="country:CH")
    json_out = response.json
    if 'state' in json_out.keys():
        npo_data_.loc[indx,'Canton'] = json_out['state']

In [22]:
npo_cantons = pd.DataFrame(npo_data_.groupby('Canton').apply(lambda x: x['Funding'].sum()))
npo_cantons.columns = ['Funding']
npo_cantons.reset_index(level=0, inplace=True)
npo_cantons.drop(0, inplace=True)

In [102]:
extended_data = pd.concat([extended_data, npo_cantons])
extended_data = pd.DataFrame(extended_data.groupby('Canton').apply(lambda x: x['Funding'].sum()))

extended_data.columns = ['Funding']
extended_data.sort_values(by='Funding', ascending=False,inplace=True)
extended_data.reset_index(level=0, inplace=True)
extended_data['Funding'] /= 1e6 
#extended_data['Funding'] = np.log(extended_data['Funding'])

extended_data.columns = ['Canton', 'Funding(in Millions CHF)']
extended_data


,Canton,Funding(in Millions CHF)
0,ZH,3480.466411
1,VD,2404.774253
2,GE,1874.051408
3,BE,1571.775507
4,BS,1387.067162
5,FR,465.671775
6,NE,424.686482
7,TI,186.099676
8,SG,168.770176
9,LU,60.314712


#### Visualization
Finally, with all the data collected we can finally perform the visualization with the help of folium. We plot the map using the TopoJSON file (with coordinates for each canton) and using the dataframe containing grant money for each canton.

In [106]:
topo_path = 'ch-cantons.topojson.json'

m = folium.Map(location=[46.76, 8.26], zoom_start=8, tiles='Mapbox Bright')

m.choropleth(geo_path=topo_path, data=extended_data, columns=['Canton', 'Funding(in Millions CHF)'], 
             key_on='feature.id', topojson='objects.cantons', fill_color='YlGn', legend_name = 'Funding (CHF)')

/Users/nesara/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.


In [107]:
m.save('m.html') #To later see in a browser.

In [108]:
m